In [1]:
import pymongo
import re
import pprint
import json
import jieba
from collections import Counter
import time
import math
import logging
from gensim.models import word2vec
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

D:\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


# 將資料由pymongo讀取近來

In [2]:
#登入pymongo
clinet=pymongo.MongoClient("mongodb://10.120.27.23:27017/")

In [3]:
#確認裡面有哪幾個database
clinet.database_names()

ServerSelectionTimeoutError: 10.120.27.23:27017: [WinError 10061] 無法連線，因為目標電腦拒絕連線。

In [ ]:
#選擇rawData這個db
db=clinet['rawData']

In [ ]:
#確認裡面有哪幾個collection
db.collection_names()

In [ ]:
#將愛評網讀入
ipeenjson=list(db.Tainan_final.find({}))

In [ ]:
ipeenjson[0]

# 將結巴字典加入自己的許多詞

In [ ]:

#結巴 匯入自己的詞典
jieba.set_dictionary('D:/WordLibrary/JiebaUse/jieba_dict.txt.big')

#停止詞字典stopwordset
stopwordset = set()
with open('D:/WordLibrary/JiebaUse/stopwords.txt', 'r', encoding='utf-8') as sw:
    for line in sw:
        stopwordset.add(line.strip('\n'))
        
#店家名稱字典dienlist
dienlist = [dien["name"] for dien in ipeenjson]

#店家種類字典stylelist
stylelist = [dien["style"] for dien in ipeenjson]

#店家種類字典introductionlist
introductionlist = [dien["introduction"] for dien in ipeenjson]

#店家種類字典coordinatelist
coordinatelist = [dien['coordinate'] for dien in ipeenjson]

#自製餐廳相關字字典pinlist
with open("D:\Data\csvtest\MyDict.csv") as f:
    mydict=f.read()
#linewords pindict每個字的評分dict
linewords=[{myword:float(line.split(",")[0]) for myword in line.split(",")[1:]}\
           for line in mydict.split("\n") if line.split(",")[0]!=""]
pindict={}
for word in linewords:
    pindict.update(word)
pinlist=list(pindict.keys())

#將以上字典都加入斷詞行列
for i in dienlist:
    jieba.add_word(i)
for i in stylelist:
    jieba.add_word(i)
for i in pinlist:
    if i !="":
        jieba.add_word(i)
        
#店家評論總數Ncommentlist
Ncommentlist = [len(i["comment"]) for i in ipeenjson]


    

In [ ]:
#各店家評論diencommentlist
diencommentlist=[[j["content"] for j in i["comment"]] for i in ipeenjson]
diencommentlist[0]

In [ ]:
#各店家回覆dienmessagelist
dienmessagelist=[[''.join(j["message"]) for j in i["comment"] if j["message"]!=None and j["message"]!='null'] for i in ipeenjson]
dienmessagelist[0]

In [ ]:
#回覆數量
Nmessagelist=[len(dienmessage) for dienmessage in dienmessagelist]

In [ ]:
#全部回覆+全部評論
dienallpinlist=[diencomment+dienmessage for diencomment,dienmessage in zip(diencommentlist,dienmessagelist)]
dienallpinlist[0]


In [ ]:
# for megs in dienmessagelist:
#     for meg in megs:
#         if meg!=None and meg!='null'and meg!='':
#             print(" ".join(meg))

#把所有回覆合併
messagecombinlist=[" ".join(["".join(meg) for meg in megs if meg!=None and meg!='null'and meg!='']) for megs in dienmessagelist]
messagecombinlist[0]

In [ ]:
#把所有評論合併
commentcombinelist = [" ".join([j["content"] for j in i["comment"]]) for i in ipeenjson]
commentcombinelist[0]

In [ ]:
#合併評論+回覆
pincombinelist=[comment+" "+message for comment,message in zip(commentcombinelist,messagecombinlist)]
pincombinelist[0]

In [ ]:
#清一下記憶體
import gc
gc.collect()


# 將許多可能會用來分析的項目做成list

# 做TD-IDF分析

In [ ]:

st=time.time()
#取得各店家的斷完詞評論pincutlist
pincutlist = []
cutcount=0
for pincombine in pincombinelist:
    cutcount+=1
    if cutcount%200==0:
        print("以切"+str(cutcount)+"篇文章的詞")
    words = jieba.cut(pincombine, cut_all=False)
    pincut=" ".join([word for word in words if word not in stopwordset and
                     '\u4e00' <= word and word <= '\u9fff' ])#and word not in dienlist and word not in pinlist
    pincutlist.append(pincut)
    
#所有店家的用詞加總前三百wordCountlist300
wordCountlist=[Counter(pincut.split(" ")).most_common(300) for pincut in pincutlist]

#TF計算 (TF=這個字出現的次數/這篇文章的總字數)
#(sum(dict(wordCount).values())=>一篇所有字的字數  wordkv[1]>>該文字字數  wordkv[0]>>該文字)
#TFList每篇文章的詞 的詞頻
TFList=[[((wordkv[0],wordkv[1]/sum(dict(wordCount).values()))) for wordkv in wordCount] for wordCount in wordCountlist]


#全文章共用了幾個詞(set可以去除重複，可迭代)
wordlist=[]
for wordCount in wordCountlist:
    for word in wordCount:
        wordlist.append(word[0])
wordlist=set(wordlist)

#nDien所有的文章(店家)數
ndien=len(dienlist)

#wordappear計算每個字出現於 (幾篇) 文章
wordappear={}
appearcount=0
for word in wordlist:
    appearcount+=1
    if appearcount%1000==0:
        print("以計算"+str(appearcount)+"個文字的出現篇數")
    n=0
    for pincut in pincutlist:
        if word in pincut:
            n+=1
    wordappear[word]=n
    
#IDF(逆向檔案頻率)=某詞  所有文章數/在幾篇文章出現 開log10 >>次方數為所
IDFlist={word:math.log(ndien/wordappear[word],10) for word in wordappear}

#dienTF每家店的全字TF值 wordTF每個詞的TF   wordTF[0]詞 wordTF[1]詞的TF值  IDFlist[wordTF[0]]詞的IDF值
TF_IDFlist=[Counter({wordTF[0]:wordTF[1]*IDFlist[wordTF[0]]for wordTF in dienTF})for dienTF in TFList]
for name,TFIDF in zip(dienlist,TF_IDFlist):
    if name in TFIDF:
        del TFIDF[name]


et=time.time()
print(et-st)

# 將上面的斷詞文章combine在一起 做word2vec model

In [ ]:


#將所有的斷詞文章combine在一起，變成超大斷詞字串，並存起來
contentcutcombine=" ".join(pincutlist)
with open('D:\Data\word2VecData\TainanContent.txt','w',encoding="utf-8") as w:
    w.write(contentcutcombine)
    
#將存起來的文章用word2vec.Text8Corpus切成有順序的詞彙list(sentences)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
sentences = word2vec.Text8Corpus("D:/Data/word2VecData/TainanContent.txt")

#將有順序的詞彙list丟給word2vec做訓練
model = word2vec.Word2Vec(sentences, size=300,window=6, min_count=4, workers=4,sg=1)
# sentences:當然了，這是要訓練的句子集，沒有他就不用跑了
# size:這表示的是訓練出的詞向量會有幾維
# alpha:機器學習中的學習率，這東西會逐漸收斂到 min_alpha
# sg:sg=1表示採用skip-gram(由本字去測他字 擴散),sg=0表示採用cbow(由他字測本字 集中)
# window:能往左往右看幾個字的意思
# workers:執行緒數目，除非電腦不錯，不然建議別超過 4
# min_count:若這個詞出現的次數小於min_count，那他就不會被視為訓練對象

# Save our model.
model.save("D:/Data/word2VecData/Tainan.model.bin")


In [ ]:
gc.collect()

# 試玩word2vec的model  需解除#

In [ ]:
# from gensim.models import word2vec
# from gensim import models
# import logging

# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
# model = models.Word2Vec.load('D:/Data/word2VecData/Tainan.model.bin')

# print("提供 3 種測試模式\n")
# print("輸入一個詞，則去尋找前一百個該詞的相似詞")
# print("輸入兩個詞，則去計算兩個詞的餘弦相似度")
# print("輸入三個詞，進行類比推理")

# while True:
#     try:
#         query = input()
#         q_list = query.split()

#         if len(q_list) == 1:
#             print("相似詞前 30 排序")
#             res = model.most_similar(q_list[0],topn = 30)
#             for item in res:
#                 print(item[0]+","+str(item[1]))

#         elif len(q_list) == 2:
#             print("計算 Cosine 相似度")
#             res = model.similarity(q_list[0],q_list[1])
#             print(res)
#         else:
#             print("%s之於%s，如%s之於" % (q_list[0],q_list[2],q_list[1]))
#             res = model.most_similar([q_list[0],q_list[1]], [q_list[2]], topn= 20)
#             for item in res:
#                 print(item[0]+","+str(item[1]))
#         print("----------------------------")
#     except Exception as e:
#         print(repr(e))


# 試Rocchio、bayes、KNN、SVM分類 (仿白話大數據寫法)

In [ ]:
#取得各店家的斷完詞評論contentcutlist
#店家種類字典stylelist
# from sklearn.datasets import fetch_20newsgroups
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer

#斷詞function
def cut_text_list(textlist):
    textcutlist=[]
    for text in textlist:
        cutwords=jieba.cut(text)
        cuttext=" ".join([word for word in cutwords if word not in stopwordset and '\u4e00' <= word <= '\u9fff'])
        textcutlist.append(cuttext)
    return textcutlist

#文字轉向量矩陣
count_vect=CountVectorizer()
X_train_counts=count_vect.fit_transform(pincutlist)
print('文字轉向量矩陣')

#向量矩陣做tfidf

tfidf_transformer=TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print('tfidf')



#ptt測試集|
with open(r'D:\Data\testData\foodptt.json') as f:
    testdatas=json.load(f)
docs_new=cut_text_list(testdatas)
X_new_counts= count_vect.transform(docs_new)
x_new_tfidf=tfidf_transformer.transform(X_new_counts)

#試用Roccio分群
# from sklearn.neighbors.nearest_centroid import NearestCentroid
st=time.time()
clf = NearestCentroid().fit(X_train_tfidf,stylelist)
predicted =clf.predict(x_new_tfidf)
print("Roccio")
pprint.pprint(predicted)
ed=time.time()
print(ed-st)


#試用bayes分群
# from sklearn.naive_bayes import MultinomialNB
st=time.time()
clf = MultinomialNB().fit(X_train_tfidf,stylelist)
predicted =clf.predict(x_new_tfidf)
print("bayes")
pprint.pprint(predicted)
ed=time.time()
print(ed-st)

#試用KNN分群
# from sklearn.neighbors import KNeighborsClassifier
st=time.time()
clf = KNeighborsClassifier(15).fit(X_train_tfidf,stylelist)
predicted =clf.predict(x_new_tfidf)
print("KNN")
pprint.pprint(predicted)
ed=time.time()
print(ed-st)

#試用svm分群
# from sklearn import svm
st=time.time()
clf = svm.SVC(kernel="linear").fit(X_train_tfidf,stylelist)
predicted =clf.predict(x_new_tfidf)
print("svm")
pprint.pprint(predicted)
ed=time.time()
print(ed-st)

# 自己肉眼辨識的答案是 合菜 美式 義式 西點 鍋類 冰品 臭豆腐 其他 義式 美式 小吃 泰式 台菜

# 最準的是 Roccio > svm > KNN > bayes，但因為Roccio有些店家無法分類，因此以SVM為準

In [ ]:
#參考PTT斷詞文章
docs_new

In [ ]:
dienallpinlist[0]

In [ ]:
gc.collect()

# 試者用自己的字典給店家分數(一篇評論的正負評價詞最多算5個)

In [ ]:
#各店家評論diencommentlist
#取得各店家的斷完詞評論contentcutlist
#自製餐廳相關字字典pinlist
#pindict每個字的評分dict
#各店家評論diencommentlist

#pinCountMaxper5list 一個店家出現的餐廳相關詞數量(一個篇評論的相同詞最多給5個分數)
st=time.time()
pinCountMaxper5list = []
countN=0
for dienallpin in dienallpinlist:
    dienWordcount={}
    for allpin in  dienallpin:
        countN+=1
        if countN%100==0:
            print("已算完"+str(countN)+"篇情緒字")
        for pinword in pinlist:
            if len(re.findall(pinword,allpin))>0:
                if len(re.findall(pinword,allpin))>5:
                    npinword=5
                else:
                    npinword=len(re.findall(pinword,allpin))
                if pinword not in dienWordcount:
                    dienWordcount[pinword]=0
                dienWordcount[pinword]+=npinword
    pinCountMaxper5list.append(dienWordcount)
ed=time.time()
print(ed-st)

In [ ]:
pinCountMaxper5list[50]

In [ ]:
#店家評論總數Ncommentlist
#店家回覆總數Nmessage
#將六篇回覆視為一篇文章，做正規化(防止評論數過多的店家分數極端化)
Npartpin=[int(Nmessage/6)+Ncomment for Nmessage,Ncomment in zip(Nmessagelist,Ncommentlist)]


#大致計算每家店的分數dienscorelist
dienscorelist=[]
for pinCount,N in zip(pinCountMaxper5list,Npartpin):
    dienscore=0
    for pin in pinCount:
        dienscore+=pinCount[pin]*pindict[pin]/N
    dienscorelist.append(dienscore)

In [ ]:
dienscorelist[2]

In [ ]:
#mydict為自己的字典的資料
#格式為:
######否開頭tag 為 非否tag 反義
#給予正分數 tag評價詞(非否評價) 同意字群同意字群同意字群同意字群同意字群同意字群
#給予負分數 tag評價詞(否開頭評價) 同意字群同意字群同意字群同意字群同意字群同意字群
#給予正分數 tag評價詞(非否評價) 同意字群同意字群同意字群同意字群同意字群同意字群
#給予負分數 tag評價詞(否開頭評價) 同意字群同意字群同意字群同意字群同意字群同意字群

#line.split(",")[1]為tag!!! word為與tag對應的同義字
synonyms=[{word:line.split(",")[1] for word in line.split(",")[1:]} for line in mydict.split("\n") if line.split(",")[0]!=""]

pinsynonymsdict={}
for synonym in synonyms:
    pinsynonymsdict.update(synonym)
del pinsynonymsdict[""]




In [ ]:
pinsynonymsdict

In [ ]:
#每篇文章pinCountMaxper5list評價詞出現次數
#評價同義詞pinsynonymsdict

#取得各餐廳的評價特徵分數featurescores!!!
featurescores=[]

#將評價詞加總及評論次數做iterate
for pinCountMaxper5,N in zip(pinCountMaxper5list,Npartpin):
    dienfeaturescore={}
    
    # 評價詞出現次數/評論次數 做為分數，其中否字開頭給負分
    for pin in pinCountMaxper5:
        if pin !="":
            if pinsynonymsdict[pin][0]=="否":
                if pinsynonymsdict[pin][1:] not in dienfeaturescore:
                    dienfeaturescore[pinsynonymsdict[pin][1:]]=0
                dienfeaturescore[pinsynonymsdict[pin][1:]]-=round(pinCountMaxper5[pin]/N,2)*1
            else:
                if pinsynonymsdict[pin] not in dienfeaturescore:
                    dienfeaturescore[pinsynonymsdict[pin]]=0
                dienfeaturescore[pinsynonymsdict[pin]]+=round(pinCountMaxper5[pin]/N,2)
    if len(dienfeaturescore)==0:
        dienfeaturescore["無評"]=1
    featurescores.append(dienfeaturescore)
    
    
    

# 將上述屬性結合成一個大分析表

In [ ]:
BigAnalyzeTable=[]
for name,style,Ncomment,contentcut,wordCount,TFIDF,pinCountMax5,score,featurescore,introduction,coordinate in zip(dienlist,stylelist,Npartpin,pincutlist,wordCountlist,TF_IDFlist,pinCountMaxper5list,dienscorelist,featurescores,introductionlist,coordinatelist):
    dien={}
    dien['name']=name
    dien['style']=style
    dien['Ncomment']=Ncomment
    dien['contentcut']=contentcut
    dien['wordCount']=wordCount
    dien['TF_IDF']=TFIDF
    dien['pinCountMaxper5']=pinCountMax5
    dien['score']=score
    dien['featurescores']=featurescore
    dien['tags']=[tag[0] for tag in Counter(TFIDF).most_common(5)]
    
    dien['introduction']=introduction
    dien['coordinate']=coordinate
    
    
    BigAnalyzeTable.append(dien)


# dienlist
# stylelist
# Ncommentlist
# pincutlist
# wordCountlist
# TF_IDFlist
# pinCountMaxper5list
# dienscorelist
# featurescores


#店家種類字典introductionlist
# introductionlist = [dien["introduction"] for dien in ipeenjson]

#店家種類字典coordinatelist
# coordinatelist = [dien['coordinate'] for dien in ipeenjson]

In [ ]:
gc.collect()

# 將每個自定義的評價詞做正規化評分、並且將分數達標者做為標籤貼上!

In [ ]:
#算出每個評論的評論數N時的 最大值b 最小值g for正規化


#allpin=評論詞list
allpin=set([line.split(",")[1] for line in mydict.split("\n") if line.split(",")[0]!="" and line.split(",")[1][0]!="否"])
standard={}

#standard為正規化所需用的值
for pin in allpin:
    for rang in [(0,5),(5,10),(10,15),(15,100)]:
        get={}
        values=[dien['featurescores'][pin] for dien in BigAnalyzeTable if pin in dien['featurescores'] and dien["Ncomment"]>rang[0] and dien["Ncomment"]<=rang[1]]
        b=0
        g=0
        for value in values:
            if value <b:
                b=round(value,1)
            if value >g:
                g=round(value,1)
        get[pin+str(rang[0])]=((g+b)/2,g-(g+b)/2)
        standard.update(get)
        print(pin+"+"+str(rang[0])+"\t"+str(g)+"\t"+str(b))

In [ ]:
standard

In [ ]:
#進行正規化的處理
for dien in BigAnalyzeTable:
    N=dien['Ncomment']
    for feature in dien['featurescores']:
        if feature !="無評":
            if N<6:
                dien['featurescores'][feature]=round((dien['featurescores'][feature])/standard[feature+str(0)][1],2)
            elif N<11 and N>=6:
                dien['featurescores'][feature]=round((dien['featurescores'][feature])/standard[feature+str(5)][1],2)
            elif N<16 and N>=11:
                dien['featurescores'][feature]=round((dien['featurescores'][feature])/standard[feature+str(10)][1],2)
            else:
                dien['featurescores'][feature]=round((dien['featurescores'][feature])/standard[feature+str(15)][1],2)

# 正規化後，每個屬性的評價區間都為2! 但未避免有些沒被負評的店判斷成負評，不將中間值移至0的位置

In [ ]:


allpin=[line.split(",")[1] for line in mydict.split("\n") if line.split(",")[0]!="" and line.split(",")[1][0]!="否"]
standard={}

for pin in allpin:
    for rang in [(0,5),(5,10),(10,15),(15,100)]:
#         get={}
        values=[dien['featurescores'][pin] for dien in BigAnalyzeTable if pin in dien['featurescores'] and dien["Ncomment"]>rang[0] and dien["Ncomment"]<=rang[1]]
        b=0
        g=0
        for value in values:
            if value <b:
                b=round(value,1)
            if value >g:
                g=round(value,1)
#         get[pin+str(rang[0])]=((g+b)/2,g-(g+b)/2)
#         standard.update(get)
        print(pin+"+"+str(rang[0])+"\t"+str(g)+"\t"+str(b))

# 貼餐廳評價標籤!

In [ ]:
pipin=[('食物美味','食物不好吃',"非常好吃",'非常不好吃'),('划算','價格較高昂',"非常便宜",'非常貴')
       ,('環境好','環境比較不好',"環境超棒",'環境很差'),('服務好','服務較差',"服務一級棒",'服務很差')
       ,('其他正面情緒','評論有些負面情緒詞',"網路評論正面情緒非常多",'網路評論負面情緒很多')]
moods=[mood[0] for mood in pipin]
for dien in BigAnalyzeTable:
    newtag=[]
    for feature in dien["featurescores"]:
        if feature in moods:
            if dien["featurescores"][feature]>1:
                for pinpare in pipin:
                    if feature==pinpare[0]:
                        newtag.append(pinpare[2])
            elif dien["featurescores"][feature]>0.4:
                newtag.append(feature)
            elif dien["featurescores"][feature]<-0.5:
                for pinpare in pipin:
                    if feature==pinpare[0]:
                        newtag.append(pinpare[3])
            elif dien["featurescores"][feature]<0.0:
                for pinpare in pipin:
                    if feature==pinpare[0]:
                        newtag.append(pinpare[1])
        else:
            if dien["featurescores"][feature]>0.7:
                newtag.append(feature)
                    
                    
#用set去除重複標籤
    dien['tags']=list(set(dien['tags']+newtag))
    print(dien['name'])
#     print(newtag)
    dien['tags'].append(dien['style'])
    print(dien['tags'])

In [ ]:
for i in BigAnalyzeTable:
    if i['score']>30 and i['Ncomment']>10:
        print(i['name']+' '+str(i['score']))
#         print(i['tags'])
        print(i['featurescores'])
        print('\n')
gooddien=[i for i in BigAnalyzeTable if i['score']>30 and i['Ncomment']>10]
# len(gooddien)

In [ ]:
with open('D:/Data/JsonData/TainanFood/bigtable_1.3.json','w') as f:
    json.dump(BigAnalyzeTable,f)

In [1]:
import json
with open('D:/Data/JsonData/TainanFood/bigtable_1.3.json') as f:
    data=json.load(f)

In [2]:
[dien['coordinate'] for dien in data]

[{'lat': 22.9999252, 'lng': 120.1534308},
 {'lat': 22.9841793, 'lng': 120.2185374},
 {'lat': 22.9903124, 'lng': 120.1981439},
 {'lat': 22.9973997, 'lng': 120.1991831},
 {'lat': 22.9980996, 'lng': 120.1741423},
 {'lat': 22.989468, 'lng': 120.162831},
 {'lat': 22.9910369, 'lng': 120.2214746},
 {'lat': 22.9890556, 'lng': 120.2041456},
 {'lat': 23.0004936, 'lng': 120.2068603},
 {'lat': 22.9891486, 'lng': 120.1951976},
 {'lat': 22.9945217, 'lng': 120.1971865},
 {'lat': 22.994548, 'lng': 120.2133713},
 {'lat': 22.9976135, 'lng': 120.1989103},
 {'lat': 23.0518138, 'lng': 120.1646187},
 {'lat': 22.9942969, 'lng': 120.2001762},
 {'lat': 22.973827, 'lng': 120.220684},
 {'lat': 22.9982344, 'lng': 120.174619},
 {'lat': 22.9933486, 'lng': 120.1976249},
 {'lat': 23.02912, 'lng': 120.2518763},
 {'lat': 22.9889141, 'lng': 120.1880278},
 {'lat': 22.9981748, 'lng': 120.1743296},
 {'lat': 22.9964763, 'lng': 120.2030135},
 {'lat': 22.9892031, 'lng': 120.1949238},
 {'lat': 22.9987503, 'lng': 120.1696348},


In [ ]:
from collections import Counter
Counter(dict(data[0]['wordCount'])).most_common(50)

In [ ]:
dict(data[0]['wordCount'])